In [61]:
import numpy as np
import pandas as pd
from datetime import datetime

In [62]:
fecha = '2024_03_10'

Cargando los datos

In [63]:
data_demo = pd.read_csv(f"./data/staging/data_demografica_{fecha}_staging.csv")
data_comportamiento = pd.read_csv(f"./data/staging/data_comportamiento_{fecha}_staging.csv")
data_cc = pd.read_csv(f"./data/staging/data_cc_{fecha}_staging.csv")

In [64]:
pd.merge(data_comportamiento, data_demo, on = 'id_distribuidor').tipo_prestamo.value_counts()

linea                     28619
prestamo_personal_red     14919
prestamo_personal_club      979
Name: tipo_prestamo, dtype: int64

In [65]:
pd.merge(data_comportamiento, data_cc, on = 'id_distribuidor').tipo_prestamo.value_counts()

linea                     16483
prestamo_personal_red      7713
prestamo_personal_club      614
Name: tipo_prestamo, dtype: int64

In [66]:
pd.merge(data_comportamiento, pd.merge(data_demo, data_cc, on = 'id_distribuidor'), on = 'id_distribuidor').tipo_prestamo.value_counts()

linea                     16438
prestamo_personal_red      7709
prestamo_personal_club      612
Name: tipo_prestamo, dtype: int64

In [67]:
data_final = pd.merge(pd.merge(data_comportamiento, data_demo, on = 'id_distribuidor'), data_cc, on = 'id_distribuidor')

In [68]:
# Quitando los replicados tomando si una persona tiene un rendimiento malo, que lo tenga siempre
#data_final = data_final.sort_values(by = 'perdida_cosecha').drop_duplicates(subset=['id_distribuidor'], keep='last')

Quitando los ids y pasando a dummy

In [69]:
data_final = data_final.drop(['id_distribuidor', 'id_red', 'id_operacion', 'folio'], axis = 1)

In [70]:
data_final_guardar = pd.concat([pd.get_dummies(data_final.drop("tipo_prestamo", axis = 1)), data_final["tipo_prestamo"]], axis = 1)

In [71]:
data_final_guardar.tipo_prestamo.value_counts()

linea                     16438
prestamo_personal_red      7709
prestamo_personal_club      612
Name: tipo_prestamo, dtype: int64

In [75]:
data_final_guardar.head(1)

,perdida_cosecha,dependientes,tiempo_vivienda,ingreso_neto,tipo_negocio,tiempo_op_negocio,edad,creditos_con_claves_prevencion,creditos_completados_con_atraso,saldo_vencido_total,...,estado_civil_viuda,tipo_vivienda_familiar,tipo_vivienda_otro,tipo_vivienda_propia,tipo_vivienda_rentada,sexo_femenino,sexo_masculino,experiencia_no,experiencia_si,tipo_prestamo
0,0.0,2.922441,-0.840328,0.001789,-0.840328,0.331977,-0.331258,0.245614,0.105263,-0.116507,...,0,1,0,0,0,1,0,1,0,prestamo_personal_red


In [77]:
data_final_guardar.to_csv(f"./data/master/data_{fecha}_master.csv", index=False)